In [1]:
#!/usr/bin/env python3
from traitlets.config.manager import BaseJSONConfigManager
cm = BaseJSONConfigManager()
cm.update(
    "rise",
    {
        "transition": "zoom",
        "start_slideshow_at": "selected",
        "footer": "<h3>Python Girona 2018</h3>"
    }
)

{'transition': 'zoom',
 'start_slideshow_at': 'selected',
 'footer': '<h3>Python Girona 2018</h3>'}

# Docker 101

## What is a Container?

A `container` embeds an application / service and all of its dependencies in a single space.

- Isolation

- Ready to run

- Standardization //run everywhere

## What is Docker?

`Docker` is a platform for developers and sysadmins to develop, ship, and run applications

- `Docker Engine`: open source containerization technology

- `Docker Hub`: SaaS service for sharing and managing app stacks

## Docker vs VM?

![alt text](ct_vs_vm.png "Containers vs VMs")



images from https://www.docker.com/resources/what-container


## Benefits

- Smaller costs (by default)

- Fast delivery
  - Compatibility
  - Maintainability
  - CI

- Rapid deployment

- Scalability

- Security